## Вариант 1

In [10]:
import requests
import json
import pickle
from random import choice
from time import sleep
import re
from bs4 import BeautifulSoup as bs
import pandas as pd

### Берем список headers

In [2]:
with open('headers.pkl', 'rb') as file:
    header_list = pickle.loads(file.read())

In [3]:
def req(vacancy_name, url, params, header_list=header_list):
    headers = choice(header_list)
    try:
        if params != None:
            return requests.get(url, params=params, headers=headers)
        else:
            return requests.get(url, headers=headers)
    except Exception as err:   
        cnt = 5
        slep = cnt
        print()
        for i in range(slep):
            print(f'\rОшибка {err}, сплю {cnt} секунд...', end ='')
            sleep(1)
            cnt -= 1
        print('Еще попытка...')
        req(vacancy_name, url, params, header_list=header_list)

In [4]:
def parsing_page(soup):
    global url
    data_of_vacancies = []
    site = url.replace('https://', '').split('/')[0]  # Наименование сайта
    vacancies = soup.find_all('div', {'class': 'vacancy-serp-item'})  # Все вакансии
    for vacancy in vacancies:
        min_salary = None
        max_salary = None
        vacancy_name = vacancy.find('a', {'class': 'bloko-link'}).getText()  # Наименование вакансии
        vacancy_link = vacancy.find('a', {'class': 'bloko-link'}).get('href')  # Ссылка
        try:
            vacancy_employer = vacancy.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).getText()  # Работодатель
        except:
            vacancy_employer = None
        try:
            salary = vacancy.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).getText() # Зарплата
            if 'от' in salary:
                min_salary = int(''.join(re.findall(r'\d+', salary)))
            elif '–' in salary:
                salary = salary.split('–')
                min_salary = int(''.join(re.findall(r'\d+', salary[0])))
                max_salary = int(''.join(re.findall(r'\d+', salary[1])))
            elif 'до' in salary:
                max_salary = int(''.join(re.findall(r'\d+', salary)))
        except:
            pass
        data_of_vacancy = {'Наименование вакансии': vacancy_name,
                           'Работодатель': vacancy_employer,
                           'Минимальная зарплата': min_salary,
                           'Максимальная зарплата': max_salary,
                           'Сайт': site,
                           'Ссылка': vacancy_link}
        data_of_vacancies.append(data_of_vacancy)
    return pd.DataFrame(data_of_vacancies)

### Наименование вакансии

In [ ]:
vacancy_name = 'python'  

### Основной цикл

In [9]:
site_url = 'https://hh.ru'
second_path_url = '/search/vacancy'
start_url = site_url + second_path_url
url = start_url  # Начальная ссылка
params = {'fromSearchLine': True,
              'st': 'searchVacancy',
              'items_on_page': '20',
              'text': vacancy_name} 
#                                  [Запрос первой страницы]
print('\rСтраница 1', end ='')
response = req(vacancy_name, url, params)
soup = bs(response.text, 'html.parser')
# Парсинг полученной страницы
result = parsing_page(soup)

# Получение ссылки на следующую страницу
try:
    next_page = soup.find('div', {'data-qa': 'pager-block'}).find('a', {'data-qa': 'pager-next'}).get('href')
    next_page = site_url + next_page
except:
    next_page = None
#                                  [Запрос последующих страниц]
while next_page != None:
    url = next_page
    params = None
#     print(f'{url}')
    response = req(vacancy_name, url, params)
    soup = bs(response.text, 'html.parser')
    now_page = int(''.join(re.findall(r'&page=(\d+)', url)))
    print(f'\rСтраница {now_page + 1}', end='')
    result = pd.concat([result, parsing_page(soup)]).reset_index(drop=True)
    try:
        next_page = soup.find('div', {'data-qa': 'pager-block'}).find('a', {'data-qa': 'pager-next'}).get('href')
        next_page = site_url + next_page
    except:
        next_page = None
    sleep(.3) # Поспим, чтоб не заблочили быстро

print('\nГотово')    

Страница 100
Готово


In [8]:
result.fillna('')

,Наименование вакансии,Работодатель,Минимальная зарплата,Максимальная зарплата,Сайт,Ссылка
0,Middle-backend Python разработчик / программист,Центр хранения и анализа больших данных,180000.0,200000.0,hh.ru,https://hh.ru/vacancy/46861261?from=vacancy_se...
1,Python Developer,Scand,,,hh.ru,https://hh.ru/vacancy/46061485?from=vacancy_se...
2,Data Analyst,Top Remote Talent,4000.0,6500.0,hh.ru,https://hh.ru/vacancy/46870106?from=vacancy_se...
3,Python Developer,LeverX International,,,hh.ru,https://hh.ru/vacancy/46826981?from=vacancy_se...
4,Программист-разработчик AI / Data scientist / ...,Clever Technology,100000.0,150000.0,hh.ru,https://hh.ru/vacancy/46085998?from=vacancy_se...
...,...,...,...,...,...,...
1995,SecOps Engineer,"Плэйтика Бел, ИООО",,,hh.ru,https://hh.ru/vacancy/46862751?from=vacancy_se...
1996,QA Engineer (WoT Blitz),"Wargaming, Moscow",,,hh.ru,https://hh.ru/vacancy/43661587?from=vacancy_se...
1997,Embedded Software Developer,HiQo Solutions,,,hh.ru,https://hh.ru/vacancy/46826086?from=vacancy_se...
1998,Senior/Principal DevOps Engineer,Oxagile,,,hh.ru,https://hh.ru/vacancy/46856002?from=vacancy_se...


### Сохраняем результат

In [11]:
with open('result.json', 'w') as file:
    file.write(json.dumps(result.to_dict()))    